# 电力生产 2

## 目标和前提条件

本示例（是之前"电力生产 1"示例的扩展）将教您如何选择最优的发电站组合来满足24小时时间范围内的预期电力需求 – 同时可以选择使用水力发电厂来满足这些需求。

该模型是H. Paul Williams所著《数学规划中的模型构建》第五版中的第16个示例，见第271-272页和326-327页。

这是一个中级示例，我们假设您了解Python和Gurobi Python API，并且对构建数学优化模型有一定了解。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含本示例和其他示例的代码库。

---
## 问题描述

在本问题中，火力发电机组分为三个不同类型，每种类型具有不同的特征（发电量、每兆瓦时成本、启动成本等）。发电机可以开启或关闭，从关闭到开启时需要支付启动成本，当发电机开启时，发电量可以在指定的最小值和最大值之间。此外还有两个水力发电厂可用，它们也具有不同的成本和发电特征。24小时的时间范围被分为5个离散时段，每个时段都有预期的总电力需求。模型决定何时开启哪些机组，以满足每个时段的需求。该模型还考虑了储备需求，即所选发电厂必须能够在不超过其最大输出的情况下增加输出，以应对实际需求超过预测需求的情况。

有一组发电机可用于满足次日的电力需求。预期需求如下：

| 时段 | 需求（兆瓦） |
| --- | --- |
| 凌晨12点至早上6点 | 15000 |
| 早上6点至9点 | 30000 |
| 早上9点至下午3点 | 25000 |
| 下午3点至6点 | 40000 |
| 晚上6点至凌晨12点 | 27000 |

火力发电机分为三类，每类型开启时的最小和最大输出如下：

| 类型 | 可用数量 | 最小输出(MW) | 最大输出(MW) |
| --- | --- | --- | --- |
| 0 | 12 |  850 | 2000 |
| 1 | 10 | 1250 | 1750 |
| 2 | 5 | 1500 | 4000 |

使用火力发电机涉及以下成本：发电机开启时的每小时成本（在最小输出时），超出最小输出的每兆瓦时成本，以及开启发电机的启动成本：

| 类型 | 每小时成本（开启时） | 超出最小输出的每兆瓦时成本 | 启动成本 |
| --- | --- | --- | --- |
| 0 | $\$1000$ | $\$2.00$ | $\$2000$ |
| 1 | $\$2600$ | $\$1.30$ | $\$1000$ |
| 2 | $\$3000$ | $\$3.00$ | $\$500$ |

此外还有两个水力发电机可用，每个在开启时都有固定的发电量：

| 水电站 | 输出(MW) |
| --- | --- |
| A | 900 |
| B | 1400 |

使用水电站的成本略有不同。每小时成本比火力发电机的小得多。水电站的真正成本来自水库水位的降低，两个机组的降低率不同。在时间范围结束前，必须通过抽水来补充水库，这需要消耗电力。水电站也有启动成本。

| 水电站 | 每小时成本（开启时） | 启动成本 | 水库水位降低率(米/小时) |
| --- | --- | --- | --- |
| A | $\$90$ | $\$1500$ | 0.31 |
| B | $\$150$ | $\$1200$ | 0.47 |

向水库抽水每提升1米高度需要消耗3000兆瓦时电力。时间范围结束时的水库水位必须等于开始时的水位。

发电机必须满足预测需求，同时还必须有足够的储备容量来应对实际需求超过预测需求的情况。对于本模型，所选火力发电机加上水力发电机必须能够产生预测需求的115%。

应该如何选择发电机来满足预期需求，以最小化总成本？

---
## 模型构建

### 集合和索引

$t \in \text{Types}=\{0,1,2\}$: 火力发电机类型。

$h \in \text{HydroUnits}=\{0,1\}$: 两个水力发电机。

$p \in \text{Periods}=\{0,1,2,3,4\}$: 时间段。

### 参数

$\text{period_hours}_{p} \in \mathbb{N}^+$: 每个时间段的小时数。

$\text{demand}_p \in \mathbb{R}^+$: 时间段 $p$ 的总电力需求。

$\text{generators}_t \in \mathbb{N}^+$: 类型 $t$ 的火力发电机数量。

$\text{start0} \in \mathbb{N}^+$: 时间范围开始时已开启的火力发电机数量（在时间段0可用，无需支付启动成本）。

$\text{min_output}_t \in \mathbb{R}^+$: 类型 $t$ 的火力发电机的最小输出（开启时）。

$\text{max_output}_t \in \mathbb{R}^+$: 类型 $t$ 的火力发电机的最大输出。

$\text{base_cost}_t \in \mathbb{R}^+$: 类型 $t$ 的火力发电机的最小运行成本（每小时）。

$\text{per_mw_cost}_t \in \mathbb{R}^+$: 类型 $t$ 的火力发电机每增加一兆瓦的成本（每小时）。

$\text{startup_cost}_t \in \mathbb{R}^+$: 类型 $t$ 的火力发电机的启动成本。

$\text{hydro_load}_h \in \mathbb{R}^+$: 水力发电机 $h$ 的输出。

$\text{hydro_cost}_h \in \mathbb{R}^+$: 水力发电机 $h$ 的运行成本。

$\text{hydro_startup_cost}_h \in \mathbb{R}^+$: 水力发电机 $h$ 的启动成本。

$\text{hydro_height_reduction}_h \in \mathbb{R}^+$: 运行水力发电机 $h$ 时每小时的水库水位降低。

### 决策变量

$\text{ngen}_{t,p} \in \mathbb{N}^+$: 时间段 $p$ 中类型 $t$ 的开启的火力发电机数量。

$\text{output}_{t,p} \in \mathbb{R}^+$: 时间段 $p$ 中类型 $t$ 的火力发电机的总发电量。

$\text{nstart}_{t,p} \in \mathbb{N}^+$: 时间段 $p$ 中需要启动的类型 $t$ 的火力发电机数量。

$\text{hydro}_{h,p} \in [0,1]$: 表示水力发电机 $h$ 在时间段 $p$ 是否开启。

$\text{hydro_start}_{h,p} \in [0,1]$: 表示水力发电机 $h$ 是否在时间段 $p$ 启动。

$\text{height}_{p} \in \mathbb{R}^+$: 时间段 $p$ 的水库水位。

$\text{pumping}_{p} \in \mathbb{R}^+$: 时间段 $p$ 用于补充水库的电力。


### 目标函数

- **成本**: 最小化满足预测电力需求的成本（美元）。

\begin{equation}
\text{Minimize} \quad Z_{on} + Z_{extra} + Z_{startup} + Z_{hydro} + Z_{hydro\_startup}
\end{equation}

\begin{equation}
Z_{on} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{base_cost}_t*\text{ngen}_{t,p}}
\end{equation}
\begin{equation}
Z_{extra} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{per_mw_cost}_t*(\text{output}_{t,p} - \text{min_load}_t*\text{ngen}_{t,p})}
\end{equation}
\begin{equation}
Z_{startup} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{startup_cost}_t*\text{nstart}_{t,p}}
\end{equation}
\begin{equation}
Z_{hydro} = \sum_{(h,p) \in \text{HydroUnits} \times \text{Periods}}{\text{hydro_cost}_h*\text{hydro}_{h,p}}
\end{equation}
\begin{equation}
Z_{hydro\_startup} = \sum_{(h,p) \in \text{HydroUnits} \times \text{Periods}}{\text{hydro_startup_cost}_h*\text{hydro_start}_{h,p}}
\end{equation}


### 约束条件

- **可用发电机**: 使用的发电机数量必须小于或等于可用数量。

\begin{equation}
\text{ngen}_{t,p} \leq \text{generators}_t  \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **需求**: 每个时间段 $p$ 中所有类型发电机的总发电量必须满足预期需求加上抽水需求。

\begin{equation}
\sum_{t \in \text{Types}}{\text{output}_{t,p}} +
\sum_{h \in \text{HydroUnits}}{\text{hydro_load}_h*\text{hydro}_{h,p}} \geq
\text{demand}_p + \text{pumping}_p \quad \forall p \in \text{Periods}
\end{equation}

- **最小/最大发电量**: 发电量必须遵守火力发电机的最小/最大值。

\begin{equation}
\text{output}_{t,p} \geq \text{min_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

\begin{equation}
\text{output}_{t,p} \leq \text{max_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **储备**: 所选火力发电机加上水力机组必须能够满足比预测高出15%的需求。

\begin{equation}
\sum_{t \in \text{Types}}{\text{max_output}_t*\text{ngen}_{t,p}} +
\sum_{h \in \text{HydroUnits}}{\text{hydro_load}_h} \geq 1.15 * \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **火力启动**: 建立活跃火力发电机数量与启动数量之间的关系（使用 $start0$ 表示时间范围开始前的时段）

\begin{equation}
\text{ngen}_{t,p} \leq \text{ngen}_{t,p-1} + \text{nstart}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **水电启动**: 建立水力发电机状态与水电启动数量之间的关系（假设水电站在时间范围开始时处于关闭状态）

\begin{equation}
\text{hydro}_{h,p} \leq \text{hydro}_{h,p-1} + \text{hydro_start}_{h,p} \quad \forall (h,p) \in \text{HydroUnits} \times \text{Periods}
\end{equation}

- **水库水位**: 跟踪水库水位。注意最后一个时间段结束时的水位必须等于第一个时间段开始时的水位。

- 水库水位约束：水位因抽水活动而升高，因水力发电而降低。

\begin{equation}
\text{height}_{p} = \text{height}_{p-1}  + \text{period_hours}_{p}*\text{pumping}_{p}/3000 -
\sum_{h \in \text{HydroUnits}}{\text{period_hours}_{p}*\text{hydro_height_reduction}_{h}*\text{hydro}_{h,p}} \quad \forall p \in \text{Periods}
\end{equation}

- 循环约束：第一个时间段的水位必须等于最后一个时间段的水位。

\begin{equation}
\text{height}_{pfirst} = \text{height}_{plast}  + \text{period_hours}_{pfirst}*\text{pumping}_{pfirst}/3000 -
\sum_{h \in \text{HydroUnits}}{\text{period_hours}_{pfirst}*\text{hydro_height_reduction}_{h}*\text{hydro}_{h,pfirst}}
\end{equation}

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
# %pip install gurobipy

In [1]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 参数设置

ntypes = 3
nperiods = 5
maxstart0 = 5
hydrounits = 2

generators = [12, 10, 5]
period_hours = [6, 3, 6, 3, 6]
demand = [15000, 30000, 25000, 40000, 27000]
min_load = [850, 1250, 1500]
max_load = [2000, 1750, 4000]
base_cost = [1000, 2600, 3000]
per_mw_cost = [2, 1.3, 3]
startup_cost = [2000, 1000, 500]

hydro_load = [900, 1400]
hydro_cost = [90, 150]
hydro_height_reduction = [0.31, 0.47]
hydro_startup_cost = [1500, 1200]

## 模型部署

我们创建一个模型和变量。对于每个时间段，我们有：一个整数决策变量来表示每种类型的活跃发电机数量(ngen)，一个整数变量来表示每种类型需要启动的发电机数量(nstart)，一个连续决策变量来表示每种类型发电机的总发电量(output)，一个二进制决策变量来表示水力机组是否活跃(hydro)，一个二进制决策变量来表示水力机组是否需要启动(hydrstart)，一个连续决策变量来表示用于补充水库的能量(pumping)，以及一个连续决策变量来表示水库水位(height)。

In [3]:
model = gp.Model('PowerGeneration2')

ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")

hydro = model.addVars(hydrounits, nperiods, vtype=GRB.BINARY, name="hydro")
hydrostart = model.addVars(hydrounits, nperiods, vtype=GRB.BINARY, name="hydrostart")
pumping = model.addVars(nperiods, vtype=GRB.CONTINUOUS, name="pumping")
height = model.addVars(nperiods, vtype=GRB.CONTINUOUS, name="height")

Using license file c:\gurobi\gurobi.lic


接下来我们插入约束条件：

活跃发电机的数量不能超过可用发电机的数量：

In [ ]:
# 发电机数量约束

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))

火力发电机类型的总发电量取决于该类型的活跃发电机数量。

In [ ]:
# 遵守每种发电机类型的最小和最大输出限制

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

每个时间段的总发电量（火力加水力）必须满足预测需求加上抽水需求。

In [ ]:
# 满足需求

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) +
                               gp.quicksum(hydro_load[unit]*hydro[unit,period] for unit in range(hydrounits))
                               >= demand[period] + pumping[period]
                               for period in range(nperiods))

维持适当的水库水位

In [ ]:
# 水库水位

reservoir = model.addConstrs(height[period] == height[period-1] + period_hours[period]*pumping[period]/3000 -
                             gp.quicksum(hydro_height_reduction[unit]*period_hours[period]*hydro[unit,period] for unit in range(hydrounits))
                             for period in range(1,nperiods))

# 循环约束 - 结束时的水位必须等于开始时的水位
reservoir0 = model.addConstr(height[0] == height[nperiods-1] + period_hours[0]*pumping[0]/3000 -
                             gp.quicksum(hydro_height_reduction[unit]*period_hours[0]*hydro[unit,0]
                             for unit in range(hydrounits)))

所选发电机必须能够应对额外的需求。

In [ ]:
# 提供足够的储备容量

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >=
                           1.15*demand[period] - sum(hydro_load)
                           for period in range(nperiods))

连接表示活跃火力发电机的决策变量与计数启动次数的决策变量。

In [ ]:
# 启动约束

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))

连接水力决策变量与水力启动决策变量。

In [ ]:
# 水电启动约束

hydro_startup0 = model.addConstrs((hydro[unit,0] <= hydrostart[unit,0])
                                    for unit in range(hydrounits))

hydro_startup = model.addConstrs((hydro[unit,period] <= hydro[unit,period-1] + hydrostart[unit,period])
                           for unit in range(hydrounits) for period in range(1,nperiods))

目标：最小化总成本。成本由五个部分组成：运行活跃火力发电机的成本，每个机组超出最小输出的发电成本，启动火力发电机的成本，运行水力机组的成本，以及启动水力机组的成本。

In [ ]:
# 目标：最小化总成本

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

hydro_active = gp.quicksum(hydro_cost[unit]*period_hours[period]*hydro[unit,period]
                           for unit in range(hydrounits) for period in range(nperiods))

hydro_startup = gp.quicksum(hydro_startup_cost[unit]*hydrostart[unit,period]
                            for unit in range(hydrounits) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj + hydro_active + hydro_startup)

接下来，我们启动优化，Gurobi将找到最优解。

In [12]:
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 85 rows, 75 columns and 215 nonzeros
Model fingerprint: 0x1b27f759
Variable types: 25 continuous, 50 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 4e+04]
Presolve removed 18 rows and 3 columns
Presolve time: 0.00s
Presolved: 67 rows, 72 columns, 194 nonzeros
Variable types: 25 continuous, 47 integer (18 binary)

Root relaxation: objective 9.990143e+05, 28 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 999014.286    0    7          - 999014.286      -     -    0s
H    0     0                    1106190.0000 999014.286  9.69%     -    0s
H    0     0                    103

---
## 分析

24小时时间窗口内的预期电力需求可以以总成本$\$1,000,630$来满足，相比之下，不使用水电站时需要$\$1,002,540$。以下是每个时间段的详细计划。

### 机组调度

下表显示了最优解中每个时间段内各类型活跃火力发电机的数量：

In [13]:
rows = ["Thermal" + str(t) for t in range(ntypes)]
units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        units.loc["Thermal"+str(t), p] = ngen[t,p].x
units

,0,1,2,3,4
Thermal0,12.0,12.0,12.0,12.0,12.0
Thermal1,3.0,9.0,9.0,9.0,9.0
Thermal2,0.0,0.0,0.0,1.0,0.0


下面显示了为实现这个计划，每个时间段需要启动的各类型发电机数量（回想一下，模型假设在时间范围开始时有5台可用）：

In [14]:
rows = ["HydroA", "HydroB"]
hydrotable = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for p in range(nperiods):
    hydrotable.loc["HydroA", p] = int(hydro[0,p].x)
    hydrotable.loc["HydroB", p] = int(hydro[1,p].x)
hydrotable

,0,1,2,3,4
HydroA,0.0,0.0,0.0,0.0,0.0
HydroB,0.0,0.0,0.0,1.0,1.0


在调度中对水电站的使用较少 - 我们只在最后两个时间段使用B站。

下面显示为支持这种水力活动水平必须进行的抽水量

In [15]:
pumptable = pd.DataFrame(columns=range(nperiods), index=["Pumping"], data=0.0)

for p in range(nperiods):
        pumptable.loc["Pumping", p] = pumping[p].x
pumptable

,0,1,2,3,4
Pumping,815.0,0.0,950.0,0.0,350.0


有趣的是，该计划在时间段4同时运行HydroB和进行抽水。
虽然降低水电站的负荷似乎可以降低成本，但在这个模型中水电站有固定的输出。如果从经济角度来看开启水电站是合理的，那么我们就必须补充消耗的水，即使这意味着要使用一些生成的电力来进行抽水。

---
## 参考文献

H. Paul Williams, 《数学规划中的模型构建》，第五版。

Copyright © 2020 Gurobi Optimization, LLC